#Fully Convolutional Neural Networks for Image Segmentation

This notebook illustrates how to build an Fully Convolutional Neural Network for image segmentation(.

We use a [custom dataset](https://drive.google.com/file/d/0B0d9ZiqAgFkiOHR1NTJhWVJMNEU/view?usp=sharing) prepared by [divamgupta](https://github.com/divamgupta/image-segmentation-keras).

This dataset is a subsample of the [CamVid](http://mi.eng.cam.ac.uk/research/projects/VideoRec/CamVid/) dataset with only label maps for 12 classes out of the original 32 classes in the dataset.
CamVid dataset is a video dataset with labelled object semantic classes. 
The custom dataset consists of labelled frames from some of the videos in the CamVid dataset.

We are using a pretrained VGG-16 network as the starting point for the feature extraction path and FCN 8 upsampling for generating labelmaps from downsampled festure maps.

##Imports

In [ ]:
import os, re, time, json
import zipfile
import PIL.Image, PIL.ImageFont, PIL.ImageDraw
import numpy as np
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf
from matplotlib import pyplot as plt
import tensorflow_datasets as tfds
import seaborn as sns

print("Tensorflow version " + tf.__version__)

##Parameters

In [ ]:
BATCH_SIZE = 64
class_names = ['sky', 'building','column/pole', 'road', 'side walk', 'vegetation', 'traffic light', 'fence', 'vehicle', 'pedestrian', 'byciclist', 'void']
colors = sns.color_palette(None, len(class_names))


##Download Dataset

The dataset we are using is hosted in Google Drive. The following code snippets perform download files from Google Driveto our root folder.

In [ ]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/fcnn-dataset.zip \
    -O /tmp/fcnn-dataset.zip

local_zip = '/tmp/fcnn-dataset.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp/fcnn')
zip_ref.close()

##Load and Prepare Dataset

This dataset has images and label maps.
Label maps are of size **(height, width , 1)** with each location denoting the corresponding pixel's class. Classes can be in the range **[0, 12]**

In the following function we will map these pixel maps to **(height, width, 12)** with each slice along the third axis having 1 if it belongs to class corresponding to that slice's index else 0.

We also resize images to a specific size and perform **normalization** on images so that pixels fall in the range **[-1, 1]**.


In [ ]:
'''
This function maps image and segmentation masks. Images are normalized so that each pixel is in the range [-1, 1]. Images and segmentation masks are resized.
Segmentation masks are mapped from (height, width, 1) to (height, width, 12).
'''
def map_filename_to_image_and_mask(t_filename, a_filename, height=224, width=224):
  img_raw = tf.io.read_file(t_filename)
  anno_raw = tf.io.read_file(a_filename)
  image = tf.image.decode_jpeg(img_raw)
  annotation = tf.image.decode_jpeg(anno_raw)
 
  #Resize Image and segmentation mask
  image = tf.image.resize(image, (height, width,))
  annotation = tf.image.resize(annotation, (height, width,))
  image = tf.reshape(image, (height, width, 3,))
  annotation = tf.cast(annotation, dtype=tf.int32)
  annotation = tf.reshape(annotation, (height, width, 1,))
  stack_list = []

  #Map Segmentation Masks
  for c in range(len(class_names)):
    mask = tf.equal(annotation[:,:,0], tf.constant(c))
    stack_list.append(tf.cast(mask, dtype=tf.int32))
  
  annotation = tf.stack(stack_list, axis=2)

  #Normalize Images
  image = image/127.5
  image -= 1

  return image, annotation

Dataset has training and testing images.
Training Dataset has two folders, one corresponding to images, other to annotaions. Similarly test dataset also has two folders. We create the tensorflow datasets from the images in these folders.

In [ ]:
def get_dataset_slice_paths(image_dir, label_map_dir):
  image_file_list = os.listdir(image_dir)
  label_map_file_list = os.listdir(label_map_dir)
  image_paths = [os.path.join(image_dir, fname) for fname in image_file_list]
  label_map_paths = [os.path.join(label_map_dir, fname) for fname in label_map_file_list]

  return image_paths, label_map_paths

def get_training_dataset(image_paths, label_map_paths):
  training_dataset = tf.data.Dataset.from_tensor_slices((image_paths, label_map_paths))
  training_dataset = training_dataset.map(map_filename_to_image_and_mask)
  training_dataset = training_dataset.shuffle(100, reshuffle_each_iteration=True)
  training_dataset = training_dataset.batch(BATCH_SIZE)
  training_dataset = training_dataset.repeat()
  training_dataset = training_dataset.prefetch(-1)
  return training_dataset

def get_validation_dataset(image_paths, label_map_paths):
  validation_dataset = tf.data.Dataset.from_tensor_slices((image_paths, label_map_paths))
  validation_dataset = validation_dataset.map(map_filename_to_image_and_mask)
  validation_dataset = validation_dataset.batch(BATCH_SIZE)
  validation_dataset = validation_dataset.repeat()  
  return validation_dataset


In [ ]:
train_count = 367
validation_count = 101
training_image_paths, training_label_map_paths = get_dataset_slice_paths('/tmp/fcnn/dataset1/images_prepped_train/','/tmp/fcnn/dataset1/annotations_prepped_train/')
validation_image_paths, validation_label_map_paths = get_dataset_slice_paths('/tmp/fcnn/dataset1/images_prepped_test/','/tmp/fcnn/dataset1/annotations_prepped_test/')

training_dataset = get_training_dataset(training_image_paths, training_label_map_paths)
validation_dataset = get_validation_dataset(validation_image_paths, validation_label_map_paths)

In [ ]:
#@title Plot Utilities [RUN ME]
def fuse_with_pil(images):
  widths = (image.shape[1] for image in images)
  heights = (image.shape[0] for image in images)
  total_width = sum(widths)
  max_height = max(heights)

  new_im = PIL.Image.new('RGB', (total_width, max_height))

  x_offset = 0
  for im in images:
    pil_image = PIL.Image.fromarray(np.uint8(im))
    new_im.paste(pil_image, (x_offset,0))
    x_offset += im.shape[1]
  
  return new_im

def give_color_to_annotation(annotation):
  seg_img = np.zeros( (annotation.shape[0],annotation.shape[1],3) ).astype('float')
  for c in range(12):
    segc = (annotation == c)
    seg_img[:,:,0] += segc*( colors[c][0] * 255.0)
    seg_img[:,:,1] += segc*( colors[c][1] * 255.0)
    seg_img[:,:,2] += segc*( colors[c][2] * 255.0)
  return seg_img

def show_predictions(image, labelmaps, titles, iou_list, dice_score_list):
  true_img = give_color_to_annotation(labelmaps[1])
  pred_img = give_color_to_annotation(labelmaps[0])


  image = image + 1
  image = image * 127.5
  images = np.uint8([image, pred_img, true_img])

  metrics_by_id = [(idx, iou, dice_score) for idx, (iou, dice_score) in enumerate(zip(iou_list, dice_score_list)) if iou > 0.0]
  metrics_by_id.sort(key=lambda tup: tup[1], reverse=True)  # sorts in place
  
  display_string_list = ["{}: IOU: {} Dice Score: {}".format(class_names[idx], iou, dice_score) for idx, iou, dice_score in metrics_by_id]
  display_string = "\n\n".join(display_string_list) 

  plt.figure(figsize=(15, 4))

  for idx, im in enumerate(images):
    plt.subplot(1, 3, idx+1)
    if idx == 1:
      plt.xlabel(display_string)
    plt.xticks([])
    plt.yticks([])
    plt.title(titles[idx], fontsize=12)
    plt.imshow(im)


def show_annotation_and_image(image, annotation):
  new_ann = np.argmax(annotation, axis=2)
  seg_img = give_color_to_annotation(new_ann)
  
  image = image + 1
  image = image * 127.5
  image = np.uint8(image)
  images = [image, seg_img]
  
  images = [image, seg_img]
  fused_img = fuse_with_pil(images)
  plt.imshow(fused_img)


def list_show_annotation(dataset):
  ds = dataset.unbatch()
  ds = ds.shuffle(buffer_size=100)

  plt.figure(figsize=(25, 15))
  plt.title("Images And Annotations")
  plt.subplots_adjust(bottom=0.1, top=0.9, hspace=0.05)

  for idx, (image, annotation) in enumerate(ds.take(9)):
    plt.subplot(3, 3, idx + 1)
    plt.yticks([])
    plt.xticks([])
    show_annotation_and_image(image.numpy(), annotation.numpy())

# utility to display training and validation curves
def plot_metrics(metric_name, title, ylim=5):
  plt.title(title)
  plt.ylim(0,ylim)
  plt.plot(history.history[metric_name],color='blue',label=metric_name)
  plt.plot(history.history['val_' + metric_name],color='green',label='val_' + metric_name)

 

##Let's Take a Look at the Dataset

In [ ]:
list_show_annotation(training_dataset)

In [ ]:
list_show_annotation(validation_dataset)

##Define the Model

###Download vgg weights

In [ ]:
!wget https://github.com/fchollet/deep-learning-models/releases/download/v0.1/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5
vgg_weights_path = "vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5"

###Define Pooling Block of VGG

Each block in the VGG encoder has convolutional layers followed by max pooling layer which downsamples the image.

In [ ]:
'''
This functions defines a block in the VGG network.
'''
def block(x, n_convs, filters, kernel_size, activation, pool_size, pool_stride, block_name):
  for i in range(n_convs):
      x = tf.keras.layers.Conv2D(filters=filters, kernel_size=kernel_size, activation=activation, padding='same', name="{}_conv{}".format(block_name, i + 1))(x)
  x = tf.keras.layers.MaxPooling2D(pool_size=pool_size, strides=pool_stride, name="{}_pool{}".format(block_name, i+1 ))(x)
  return x


###Define VGG-16

In [ ]:
'''
This function defines the VGG encoder.
'''
def VGG_16(image_input):
  x = block(image_input,n_convs=2, filters=64, kernel_size=(3,3), activation='relu',pool_size=(2,2), pool_stride=(2,2), block_name='block1')
  p1= x

  x = block(x,n_convs=2, filters=128, kernel_size=(3,3), activation='relu',pool_size=(2,2), pool_stride=(2,2), block_name='block2')
  p2 = x

  x = block(x,n_convs=3, filters=256, kernel_size=(3,3), activation='relu',pool_size=(2,2), pool_stride=(2,2), block_name='block3')
  p3 = x

  x = block(x,n_convs=3, filters=512, kernel_size=(3,3), activation='relu',pool_size=(2,2), pool_stride=(2,2), block_name='block4')
  p4 = x

  x = block(x,n_convs=3, filters=512, kernel_size=(3,3), activation='relu',pool_size=(2,2), pool_stride=(2,2), block_name='block5')
  p5 = x

  vgg  = tf.keras.Model(image_input , p5)
  vgg.load_weights(vgg_weights_path) 

  n = 4096
  c6 = tf.keras.layers.Conv2D( n , ( 7 , 7 ) , activation='relu' , padding='same', name="conv6")(p5)
  c7= tf.keras.layers.Conv2D( n , ( 1 , 1 ) , activation='relu' , padding='same', name="conv7")(c6)
  return(p1, p2, p3, p4, c7)





###Define FCN 8 Encoder

In [ ]:
'''
This function defines the FCN 8 decoder.
'''
def fcn8_decoder(convs, n_classes):
  f1, f2, f3, f4, f5 = convs
  
  o = tf.keras.layers.Conv2DTranspose(n_classes , kernel_size=(4,4) ,  strides=(2,2) , use_bias=False )(f5)
  o = tf.keras.layers.Cropping2D(cropping=(1,1))(o)

  o2 = f4
  o2 = ( tf.keras.layers.Conv2D(n_classes , ( 1 , 1 ) , activation='relu' , padding='same'))(o2)

  o = tf.keras.layers.Add()([o, o2])

  o = (tf.keras.layers.Conv2DTranspose( n_classes , kernel_size=(4,4) ,  strides=(2,2) , use_bias=False ))(o)

  o2 = ( tf.keras.layers.Conv2D(n_classes , ( 1 , 1 ) , activation='relu' , padding='same'))(f3)

  o = tf.keras.layers.Cropping2D(cropping=(1, 1))(o)
  o = tf.keras.layers.Add()([o, o2])
     
  o = tf.keras.layers.Conv2DTranspose(n_classes , kernel_size=(8,8) ,  strides=(8,8) , use_bias=False )(o)

  o = (tf.keras.layers.Activation('softmax'))(o)


  return o

###Define Final Model

In [ ]:
'''
This function defines the final segmentation model by chaining together the encoder and decoder.
'''
def segmentation_model():
  inputs = tf.keras.layers.Input(shape=(224,224,3,))
  convs = VGG_16(image_input=inputs)
  outputs = fcn8_decoder(convs, 12)
  model = tf.keras.Model(inputs=inputs, outputs=outputs)
  return model


In [ ]:
model = segmentation_model()
model.summary()

###Compile Model

We use categorical crossentropy since we have transformed the label map to one hot encoded vectors for each pizel in the image.

In [ ]:
sgd = tf.keras.optimizers.SGD(lr=1E-2, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

##Train Model

In [ ]:
EPOCHS = 200
steps_per_epoch = train_count//BATCH_SIZE
validation_steps = validation_count//BATCH_SIZE

history = model.fit(training_dataset,
                    steps_per_epoch=steps_per_epoch, validation_data=validation_dataset, validation_steps=validation_steps, epochs=EPOCHS)

#Evaluate Model

In [ ]:
def get_images_and_segments_test_arrays():
  y_true_segments = []
  y_true_images = []
  test_count = 64

  ds = validation_dataset.unbatch()
  ds = ds.batch(101)

  for image, annotation in ds.take(1):
    y_true_images = image
    y_true_segments = annotation


  y_true_segments = y_true_segments[:test_count, : ,: , :]
  y_true_segments = np.argmax(y_true_segments, axis=3)  

  return y_true_images, y_true_segments

y_true_images, y_true_segments = get_images_and_segments_test_arrays()

###Make Predictions

In [ ]:
results = model.predict(validation_dataset, steps=validation_steps)
results = np.argmax(results, axis=3)

###Compute IOU

In [ ]:
'''
This function computes IOU and Dice Score.
'''
def compute_metrics(y_true, y_pred):
  class_wise_iou = []
  class_wise_dice_score = []

  smoothening_factor = 0.00001

  for i in range(12):
    intersection = np.sum((y_pred == i) * (y_true == i))
    y_true_area = np.sum((y_true == i))
    y_pred_area = np.sum((y_pred == i))
    combined_area = y_true_area + y_pred_area
    
    iou = (intersection + smoothening_factor) / (combined_area - intersection + smoothening_factor)
    class_wise_iou.append(iou)
    
    dice_score =  2 * ((intersection + smoothening_factor) / (combined_area + smoothening_factor))
    class_wise_dice_score.append(dice_score)

  return class_wise_iou, class_wise_dice_score

###Show Predictions and IOU

In [ ]:
#@title Visualize Output [RUN ME]
integer_slider = 63 #@param {type:"slider", min:0, max:63, step:1}
iou, dice_score = compute_metrics(y_true_segments[integer_slider], results[integer_slider])  
show_predictions(y_true_images[integer_slider], [results[integer_slider], y_true_segments[integer_slider]], ["Image", "Predicted Mask", "True Mask"], iou, dice_score)

###Display Class Wise Metrics

In [ ]:
cls_wise_iou, cls_wise_dice_score = compute_metrics(y_true_segments, results)


In [ ]:
for idx, iou in enumerate(cls_wise_iou):
  spaces = ' ' * (13-len(class_names[idx]) + 2)
  print("{}{}{} ".format(class_names[idx], spaces, iou)) 


In [ ]:
for idx, dice_score in enumerate(cls_wise_dice_score):
  spaces = ' ' * (13-len(class_names[idx]) + 2)
  print("{}{}{} ".format(class_names[idx], spaces, dice_score)) 


###Plot Loss Curves

In [ ]:
plot_metrics("loss", title="Training vs Validation Loss", ylim=2)